# analyze non-tRNA genes

## outputs

* top100_RNA_genes.tsv
* other_rna_gene_mapping_counts.filter_{filter_treshold}.tsv


In [1]:
import polars as pl


In [2]:
exclude_rna_types = ['lncRNA', "Mt_tRNA", "Mt_rRNA", ]

rna_fn = "catLiftOffGenesV1.no_chr.bed" 

rnas_df = pl.read_csv(rna_fn, 
                      columns = [3, 17, 20], 
                      comment_char="#", 
                      sep="\t", 
                      new_columns=["gene_id", "rna_class", "ensembl_id" ]
                     ).filter((pl.col("rna_class").str.ends_with("RNA")) & (~pl.col("rna_class").is_in(exclude_rna_types)))

rnas_df

gene_id,rna_class,ensembl_id
str,str,str
"""RNU6-1199P-201...","""snRNA""","""ENSG0000022318..."
"""MIR200B-201""","""miRNA""","""ENSG0000020773..."
"""MIR200A-201""","""miRNA""","""ENSG0000020760..."
"""MIR429-201""","""miRNA""","""ENSG0000019897..."
"""MIR6726-201""","""miRNA""","""ENSG0000027807..."
"""MIR6727-201""","""miRNA""","""ENSG0000028371..."
"""MIR6808-201""","""miRNA""","""ENSG0000028437..."
"""RN7SL657P-201""","""misc_RNA""","""ENSG0000026429..."
"""MIR4251-201""","""miRNA""","""ENSG0000028357..."


In [3]:
renaming_dict = {"DU145_NT_CIC.last.t2t_trnas.51bp.bam": "DU145_NT", 
                 "DU145_BoSeq_CIC.last.t2t_trnas.51bp.bam":"DU145_BoSeq",
                 "MHCC97H_Chen_NT.last.t2t_trnas.75bp.bam": "MHCC97H_NT_Chen", 
                 "MHCC97H_Chen_TRACSeq.last.t2t_trnas.75bp.bam":"MHCC97H_TRACSeq_Chen"
                }
counts_fn = "merged_CIC_Chen.no_trna_no_lnc_RNAs.tsv"
counts_df = pl.read_csv(counts_fn,  
                      comment_char="#", 
                      sep="\t",
                        dtypes = {"Chr":pl.Utf8}
                       ).with_columns(pl.col("Geneid").str.split(by="_").arr.get(-1).alias("ensembl_id"))
counts_df = counts_df.rename(renaming_dict)
counts_df.sort(["DU145_NT", "ensembl_id" ], descending=True).head(100).write_csv("top100_RNA_genes.tsv", sep="\t")

In [4]:
rnas_class_counts = rnas_df.groupby("rna_class").count().rename({"count":"annotation_counts"}).sort("rna_class")
rnas_class_counts

rna_class,annotation_counts
str,u32
"""miRNA""",2046
"""misc_RNA""",2231
"""rRNA""",1007
"""sRNA""",5
"""scRNA""",2
"""scaRNA""",48
"""snRNA""",1902
"""snoRNA""",948
"""vault_RNA""",1


In [5]:
joined_rnas_class_df = rnas_class_counts

filter_treshold = 10
cols_for_counts = ["DU145_NT", "DU145_BoSeq", "MHCC97H_NT_Chen", "MHCC97H_TRACSeq_Chen"]

for sample in cols_for_counts:
    new_count_name = f"count_{sample}"
    tmp_df = counts_df.filter(pl.col(sample) >= filter_treshold)
    tmp_df = tmp_df.join(rnas_df, on="ensembl_id").groupby("rna_class").count().rename({"count":new_count_name}) #.sort("rna_class")
    print(tmp_df)
    joined_rnas_class_df = joined_rnas_class_df.join(tmp_df, on="rna_class", how="left")
    print(joined_rnas_class_df)

joined_rnas_class_df.write_csv(f"other_rna_gene_mapping_counts.filter_{filter_treshold}.tsv", sep="\t")


shape: (7, 2)
┌───────────┬────────────────┐
│ rna_class ┆ count_DU145_NT │
│ ---       ┆ ---            │
│ str       ┆ u32            │
╞═══════════╪════════════════╡
│ misc_RNA  ┆ 242            │
│ sRNA      ┆ 1              │
│ scaRNA    ┆ 17             │
│ snoRNA    ┆ 298            │
│ miRNA     ┆ 140            │
│ rRNA      ┆ 68             │
│ snRNA     ┆ 247            │
└───────────┴────────────────┘
shape: (9, 3)
┌───────────┬───────────────────┬────────────────┐
│ rna_class ┆ annotation_counts ┆ count_DU145_NT │
│ ---       ┆ ---               ┆ ---            │
│ str       ┆ u32               ┆ u32            │
╞═══════════╪═══════════════════╪════════════════╡
│ miRNA     ┆ 2046              ┆ 140            │
│ misc_RNA  ┆ 2231              ┆ 242            │
│ rRNA      ┆ 1007              ┆ 68             │
│ sRNA      ┆ 5                 ┆ 1              │
│ ...       ┆ ...               ┆ ...            │
│ scaRNA    ┆ 48                ┆ 17             │
│ snRNA 